<a href="https://colab.research.google.com/github/venkat2ram/Keras-and-Tensorflow/blob/master/uda_L10_Text_Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#############Not able to run program due to RAM issue################

In [4]:
import tensorflow as tf
import tensorflow.keras as keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import string

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
url=tf.keras.utils.get_file('song_data.csv','https://drive.google.com/uc?id=1LiJFZd41ofrWoBtW-pMYsfz1w8Ny0Bj8')

In [7]:
dataset=pd.read_csv(url)

In [9]:
dataset=dataset[:20000]

In [10]:
def create_lyrics_corpus(dataset, field):
  # Remove all other punctuation
  dataset[field] = dataset[field].str.replace('[{}]'.format(string.punctuation), '')
  # Make it lowercase
  dataset[field] = dataset[field].str.lower()
  # Make it one long string to split by line
  lyrics = dataset[field].str.cat()
  corpus = lyrics.split('\n')
  # Remove any trailing whitespace
  for l in range(len(corpus)):
    corpus[l] = corpus[l].rstrip()
  # Remove any empty lines
  corpus = [l for l in corpus if l != '']

  return corpus

def tokenize_data(corpus,num_words=-1):
  if num_words>-1:
    tokenizer=Tokenizer(num_words=num_words)
  else:
    tokenizer=Tokenizer()
  tokenizer.fit_on_texts(corpus)
  return tokenizer


In [11]:
corpus=create_lyrics_corpus(dataset,'text')
tokenizer=tokenize_data(corpus)


In [12]:
total_words=len(tokenizer.word_index)+1

In [13]:
token_list=tokenizer.texts_to_sequences(['look at her face its a wonderful face'])[0]

In [9]:
sequences=[]
for line in corpus:
  token_list=tokenizer.texts_to_sequences([line])[0]
  for i in range(1,len(token_list)):
    n_gram_sequence=token_list[:i+1]
    sequences.append(n_gram_sequence)



In [10]:
max_sequnce_length=max([len(seq) for seq in sequences ])
padded_sequences=np.array(pad_sequences(sequences,maxlen=max_sequnce_length,padding='pre'))

In [11]:
input_sequences,labels=padded_sequences[:,:-1],padded_sequences[:,-1]

In [2]:
one_hot_labels=tf.keras.utils.to_categorical(labels,num_classes=total_words)

NameError: ignored

In [1]:
labels

NameError: ignored